<a href="https://colab.research.google.com/github/fjt067316/Multi-Class-Perceptron-Binary-Search/blob/main/multiclass_perceptron_bin_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch

In [4]:
import csv
import torch
from itertools import islice

csv_file_path = '/content/sample_data/mnist_train_small.csv'

train_labels = []
train_data = []

with open(csv_file_path, 'r') as file:
    # Create a CSV DictReader object
    csv_reader = csv.reader(file)

    # Iterate over the rows in the CSV file
    for row in islice(csv_reader, 1, None):
      if len(row) == 785:
        train_labels.append(int(row[0]))
        normalized_data = [float(value) / 255.0 for value in row[1:]] # normalize values
        train_data.append(normalized_data + [1]) # add 1 for bias

weights_bias = torch.zeros((10, 785)) # last entry is bias
train_data = torch.tensor(train_data)

In [ ]:
# Binary Search perceptron
epoch = 100
correct = 0
epoch_samples_binary = []

for i, data in enumerate(train_data):
  output = torch.matmul(weights_bias, data)

  if(i % 100 == 0):
    print(f"% corrct: { correct / epoch}")
    correct = 0
    epoch_samples_binary.append(correct / epoch)

  err: bool = False

  if output[0] < 0 and train_labels[i] in {9,8,0,6,7}: # first perceptron for binary procedure
    # incorrect classification
    correct_sign = 1
    weights_bias[0] = weights_bias[0] + data*correct_sign
    err = True
  elif output[0] >= 0 and train_labels[i] in {5,4,3,2,1}:
    # incorrect classification
    correct_sign = -1
    weights_bias[0] = weights_bias[0] + data*correct_sign
    err = True
  if output[1] < 0 and train_labels[i] in {9,8,0}:
    # incorrect classification
    correct_sign = 1
    weights_bias[1] = weights_bias[1] + data*correct_sign
    err = True
  elif output[1] >= 0 and train_labels[i] in {6, 7}:
    correct_sign = -1
    weights_bias[1] = weights_bias[1] + data*correct_sign
    err = True
  if output[2] < 0 and train_labels[i] in {9, 8}:
    # incorrect clasification
    correct_sign = 1
    weights_bias[2] = weights_bias[2] + data*correct_sign
    err = True
  elif output[2] >= 0 and train_labels[i] in {0}:
    correct_sign = -1
    weights_bias[2] = weights_bias[2] + data*correct_sign
    err = True
  if output[3] < 0 and train_labels[i] in {9}:
    # incorrect clasification
    correct_sign = 1
    weights_bias[3] = weights_bias[3] + data*correct_sign
    err = True
  elif output[3] >= 0 and train_labels[i] in {8}:
    correct_sign = -1
    weights_bias[3] = weights_bias[3] + data*correct_sign
    err = True
  if output[4] < 0 and train_labels[i] in {6}:
    # incorrect classification
    correct_sign = 1
    weights_bias[4] = weights_bias[4] + data*correct_sign
    err = True
  elif output[4] >= 0 and train_labels[i] in {7}:
    correct_sign = -1
    weights_bias[4] = weights_bias[4] + data*correct_sign
    err = True

  if output[5] < 0 and train_labels[i] in {5,4}:
    correct_sign = 1
    weights_bias[5] = weights_bias[5] + data*correct_sign
    err = True
  elif output[5] >= 0 and train_labels[i] in {3,2,1}:
    correct_sign = -1
    weights_bias[5] = weights_bias[5] + data*correct_sign
    err = True
  if output[6] < 0 and train_labels[i] in {6}:
    correct_sign = 1
    weights_bias[6] = weights_bias[6] + data*correct_sign
    err = True
  elif output[6] >= 0 and train_labels[i] in {2,1}:
    correct_sign = -1
    weights_bias[6] = weights_bias[6] + data*correct_sign
    err = True
  if output[7] < 0 and train_labels[i] in {2}:
    correct_sign = 1
    weights_bias[7] = weights_bias[7] + data*correct_sign
    err = True
  elif output[7] >= 0 and train_labels[i] in {1}:
    correct_sign = -1
    weights_bias[7] = weights_bias[7] + data*correct_sign
    err = True
  if output[8] < 0 and train_labels[i] in {5}:
    correct_sign = 1
    weights_bias[8] = weights_bias[8] + data*correct_sign
    err = True
  elif output[8] >= 0 and train_labels[i] in {4}:
    correct_sign = -1
    weights_bias[8] = weights_bias[8] + data*correct_sign
    err = True

  if not err:
    correct += 1


In [ ]:
# one vs all perceptron
# one perceptron per class, highest signal wins
# no weird class groups just k0 => digit 0
epoch = 100
correct = 0
epoch_samples_k_class = []
weights_bias = torch.zeros((10, 785)) # last entry is bias

for i, data in enumerate(train_data):
  output = torch.matmul(weights_bias, data)

  if(i % 100 == 0):
    print(f"% corrct: { correct / epoch}")
    correct = 0
    epoch_samples_k_class.append(correct / epoch)

  err: bool = False

  for idx, out in enumerate(output):

    if out >= 0 and idx != train_labels[i]:
      weights_bias[idx] += -1*data
      err = True
    elif out < 0 and idx == train_labels[i]:
      weights_bias[idx] += data
      err = True
  if not err:
    correct += 1


In [ ]:
# One vs One
# Most votes win
#
epoch = 100
correct = 0
epoch_samples_k_class = []
n_classes = 10
n_weights = int((n_classes*(n_classes-1))/2)
weights_bias = torch.zeros((n_weights, 785)) # last entry is bias

for i, data in enumerate(train_data):
  output = torch.matmul(weights_bias, data)

  if(i % 100 == 0):
    print(f"% corrct: { correct / epoch}")
    correct = 0
    epoch_samples_k_class.append(correct / epoch)

  err: bool = False

  for idx, out in enumerate(output):
    # first 9 vote for 9
    # next 8 vote for 8 etc

    if out >= 0 and idx != train_labels[i]:
      weights_bias[idx] += -1*data
      err = True
    elif out < 0 and idx == train_labels[i]:
      weights_bias[idx] += data
      err = True
    if not err:
      correct += 1